In [1]:
# Apply custon style to notebook
from IPython.core.display import HTML
import pathlib
styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
styles = open(styles_path, "r").read()
HTML(f"<style>{styles}</style>")

# 2.0 Higher Order Derivatives

$\large{\text{Manipulator Differential Kinematics}} \\ \large{\text{Part II: Acceleration and Advanced Applications}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

The sections of the Tutorial paper related to this notebook are listed next to each contents entry.
It is beneficial to read these sections of the paper before attempting the notebook Section.

### Contents

[2.1 Higher Order Derivatives](#pre)
* Part I of the Tutorial
  * Deriving the Manipulator Jacobian
  * First Derivative of an Elementary Transform
  * The Manipulator Jacobian
  * Fast Manipulator Jacobian

[2.5 Robotics Toolbox Jacobians](#rtb)

In [2]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm

# use timeit to benchmark some methods
from timeit import default_timer as timer

# ansitable is a great package for printing tables in a terminal
from ansitable import ANSITable

<br>

<a id='pre'></a>
### 2.1 Higher Order Derivatives
---


Starting from $n = 3$, obtaining the matrix of $n^{th}$ partial derivative of the manipulator kinematics can be obtained using the product rule on the $(n - 1)^{th}$ partial derivative while considering their partitioned form.

For example, to obtain the $3^{rd}$ partial derivative, we take the partial derivative of the manipulator Hessian with respect to the joint coordinates, in its partitioned form

\begin{align*}
    \frac{\partial \bf{H}_{jk}(\bf{q})}
         {\partial q_l}
    &=
    \begin{pmatrix}
        \dfrac{\partial \bf{H}_{a_{jk}}(\bf{q})}
              {\partial q_l} \\
        \dfrac{\partial \bf{H}_{\alpha_{jk}}(\bf{q})}
              {\partial q_l}
    \end{pmatrix}  \\
    &=
    \begin{pmatrix}
        \dfrac{\partial }
              {\partial q_l}
        \left( \bf{J}_{\omega_k} \times \bf{J}_{\omega_j} \right) \\
        \dfrac{\partial}
              {\partial q_l}
        \left( \bf{J}_{\omega_k} \times \bf{J}){v_j} \right)
    \end{pmatrix}  \\
    &=
    \begin{pmatrix}
        \left( \bf{H}_{\omega_kl} \times \bf{J}_{\omega_j} \right) +
        \left( \bf{J}_{\omega_k} \times \bf{H}_{\omega_jl} \right) \\
        \left( \bf{H}_{\omega_kl} \times \bf{J}_{v_j} \right) +
        \left( \bf{J}_{\omega_k} \times \bf{H}_{v_jl} \right)
    \end{pmatrix}
\end{align*}

where $\frac{\partial \bf{H}_{jk}(\bf{q})}{\partial q_l} \in \mathbb{R}^{6}$. Continuing, we obtain the following

\begin{align*}
    \frac{\partial \bf{H}_{jk}(\bf{q})}
         {\partial \bf{q}}
    &=
    \begin{pmatrix}
        \dfrac{\partial \bf{H}_{jk}(\bf{q})}
            {\partial q_0} & \cdots & 
        \dfrac{\partial \bf{H}_{jk}(\bf{q})}
            {\partial q_n}
    \end{pmatrix}
\end{align*}

where $\frac{\partial \bf{H}_{jk}(\bf{q})}{\partial q_l} \in \mathbb{R}^{6 \times n}$,

\begin{align*}
    \frac{\partial \bf{H}_{j}(\bf{q})}
        {\partial \bf{q}}
    &=
    \begin{pmatrix}
        \dfrac{\partial \bf{H}_{j_0}(\bf{q})}
            {\partial \bf{q}} & \cdots & 
        \dfrac{\partial \bf{H}_{j_n}(\bf{q})}
            {\partial \bf{q}}
    \end{pmatrix}
\end{align*}

where $\frac{\partial \bf{H}_{jk}(\bf{q})}{\partial \bf{q}} \in \mathbb{R}^{6 \times n \times n}$, and finally 

\begin{align*}
    \frac{\partial \bf{H}(\bf{q})}
         {\partial \bf{q}}
    &=
    \begin{pmatrix}
        \dfrac{\partial \bf{H}_{0}(\bf{q})}
            {\partial \bf{q}} & \cdots & 
        \dfrac{\partial \bf{H}_{n}(\bf{q})}
            {\partial \bf{q}}
    \end{pmatrix}
\end{align*}

where $\frac{\partial \bf{H}(\bf{q})}{\partial \bf{q}} \in \mathbb{R}^{6 \times n \times n \times n}$ is the 4-dimensional tensor representing the $3^{rd}$ partial derivative of the manipulator kinematics.

We have included a function as part of our open source Robotics Toolbox for Python \cite{rtb} which can calculate the $n^{th}$ partial derivative of the manipulator kinematics. Note that the function has $\mathcal{O}(n^{order})$ time complexity, where $order$ represents the order of the partial derivative being calculated.

In [51]:
def partial_fkine0(ets, q, n):
    r"""
    Manipulator Forward Kinematics nth Partial Derivative

    :return: The nth Partial Derivative of the forward kinematics
    """

    # Calculate the Jacobian and Hessian
    J = ets.jacob0(q)
    H = ets.hessian0(q)

    # A list of derivatives, starting with the jacobian and hessian
    dT = [J, H]

    # The tensor dimensions of the latest derivative
    # Set to the surrent size of the Hessian
    size = [ets.n, 6, ets.n]

    count = np.array([0, 0])

    c = 2

    def add_indices(indices, c):
        total = len(indices * 2)
        new_indices = []

        for i in range(total):
            j = i // 2
            new_indices.append([])
            new_indices[i].append(indices[j][0].copy())
            new_indices[i].append(indices[j][1].copy())

            if i % 2 == 0:
                # if even number
                new_indices[i][0].append(c)
            else:
                # if odd number
                new_indices[i][1].append(c)

        print(new_indices)

        return new_indices

    def add_pdi(pdi):
        total = len(pdi * 2)
        new_pdi = []

        for i in range(total):
            j = i // 2
            new_pdi.append([])
            new_pdi[i].append(pdi[j][0])
            new_pdi[i].append(pdi[j][1])

            # if even number
            if i % 2 == 0:
                new_pdi[i][0] += 1
            # if odd number
            else:
                new_pdi[i][1] += 1

        return new_pdi

    # these are the indices used for the hessian
    indices = [[[1], [0]]]

    # The partial derivative indices (pdi)
    # the are the pd indices used in the cross prods
    pdi = [[0, 0]]

    # The length of dT correspods to the number of derivatives we have calculated
    while len(dT) != n:

        # Add to the start of the tensor size list 
        size.insert(0, ets.n)
        count = np.concatenate(([0], count))
        indices = add_indices(indices, c)
        pdi = add_pdi(pdi)
        c += 1

        # Allocate our new partial derivative tensor
        pd = np.zeros(size)

        # We need to loop n^c times
        # There are n^c columns to calculate
        for _ in range(ets.n**c):
            
            # Allocate the rotation and translation components
            rot = np.zeros(3)
            trn = np.zeros(3)

            for j in range(len(indices)):
                pdr0 = dT[pdi[j][0]]
                pdr1 = dT[pdi[j][1]]

                idx0 = count[indices[j][0]]
                idx1 = count[indices[j][1]]

                print()
                print(pdr0.shape)
                print(pdr1.shape)


                # rot += np.cross(pdr0[(slice(3, 6), *idx0)], pdr1[(slice(3, 6), *idx1)])
                # trn += np.cross(pdr0[(slice(3, 6), *idx0)], pdr1[(slice(0, 3), *idx1)])
                # print(idx1)
                # print(pdr1[(*idx1, slice(3, 6))])

                rot += np.cross(pdr0[(*idx0, slice(3, 6))], pdr1[(*idx1, slice(3, 6))])
                trn += np.cross(pdr0[(*idx0, slice(3, 6))], pdr1[(*idx1, slice(0, 3))])

            # pd[(slice(0, 3), *count)] = trn
            # pd[(slice(3, 6), *count)] = rot
            pd[(*count, slice(0, 3))] = trn
            pd[(*count, slice(3, 6))] = rot

            count[0] += 1
            for j in range(len(count)):
                if count[j] == ets.n:
                    count[j] = 0
                    if j != len(count) - 1:
                        count[j + 1] += 1

        dT.append(pd)

    return dT[-1]

In [52]:
# Make a panda robot
panda = rtb.models.Panda()
ets = panda.ets()



In [53]:
partial_fkine0(ets, panda.qr, 3)

[[[1, 2], [0]], [[1], [0, 2]]]

(7, 6, 7)
(6, 7)

(6, 7)
(7, 6, 7)

(7, 6, 7)
(6, 7)

(6, 7)
(7, 6, 7)

(7, 6, 7)
(6, 7)

(6, 7)
(7, 6, 7)

(7, 6, 7)
(6, 7)

(6, 7)
(7, 6, 7)

(7, 6, 7)
(6, 7)

(6, 7)
(7, 6, 7)

(7, 6, 7)
(6, 7)

(6, 7)
(7, 6, 7)

(7, 6, 7)
(6, 7)


IndexError: index 6 is out of bounds for axis 0 with size 6